In [1]:
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'kagglehub'

In [ ]:

FILE_PATH = r"C:\Users\mrsmo\.cache\kagglehub\datasets\omniamahmoudsaeed\real-estate-sales-2001-2022\versions\1\Real_Estate_Sales_2001-2022_GL.csv"

# Read the CSV
df = pd.read_csv(
    FILE_PATH,
    parse_dates=["Date Recorded"],  # Parse this column as datetime
    low_memory=False                 # Prevent dtype guessing issues on large files
)

# ====================================================
# 4. Convert Columns to Category Type
# ====================================================
cat_columns = ["List Year", "Property Type", "Residential Type", "Town"]
df[cat_columns] = df[cat_columns].astype("category")

# ====================================================
# 5. Clean Up Sales Ratio Column (Remove Known Outliers)
# ====================================================
BAD_SALES_RATIO_INDICES = [
    400223, 387198, 756066, 517857, 950063, 953854, 1076552, 610694, 514136, 395137, 393010, 390655,
    247298, 417869, 474399, 221364, 478279, 251354, 423895, 433617, 863110, 134811, 397613, 479568, 666388, 811579, 413974,
    55055, 270157, 480988, 482770, 271053, 274878, 312695, 353650, 361096, 381148, 384550, 705544, 740634, 750658, 
    466006, 671159, 477350, 457875, 460176, 402630, 339155, 411821, 411133, 91602, 92379, 92547, 92576, 93312, 93480, 93922,
    93958, 94517, 94557, 94807, 95881, 95920, 96250, 98359, 97738, 96673, 96280, 100898, 100500, 98910, 98664, 102937, 101464, 101250, 103665, 
    105660, 108030, 108015, 107930, 107604, 107292, 1096079, 1096037, 1095863, 1089984, 1090576, 1091434, 1092817, 1093416,
    1094013, 1094167, 1097459
]

# Replace bad Sales Ratio values with NaN
df.loc[BAD_SALES_RATIO_INDICES, "Sales Ratio"] = np.nan


pd.set_option("display.max_rows", None)
df.head()


In [ ]:
#towns that still had sales ratios from 20 and over were excluded because the sales prices
#that cause such disproportionate sales ratios are alost always under $300,000 when people sell partial ownerships
#and it affects the lowest category
#towns_to_exclude = ["Stafford", "East Hartford", "Farmington", "East Haven"]

#df = df[~df["Town"].isin(towns_to_exclude)]

# Drop them from the category list too
#df["Town"] = df["Town"].cat.remove_unused_categories()


In [ ]:
#can be used to filter out all low sales 
#df = df[df["Sale Amount"] >= 20_000]


In [ ]:
# Define bin edges
df = df[df["Sale Amount"] >= 20_000].copy()
bins = [20_000, 300_000, 700_000, 1_000_000, float("inf")]

# Define bin labels
labels = ["Under $300k", "Mid", "$700k-$1M", "Luxury"]
# Create new column
df["Sale Price Group"] = pd.cut(
    df["Sale Amount"],
    bins=bins,
    labels=labels,
    right=False  # means "less than" on the upper bound
)

# Preview result
df[["Sale Amount", "Sale Price Group"]].head()


In [ ]:
df.groupby("Sale Price Group")["Sales Ratio"].mean()


In [ ]:
#town = df.groupby("Town")
#sales_df = town["Sales Ratio"].mean().reset_index()
#sales_df.columns = ["Town", "Avg_Sales_Ratio"]
#sales_df

In [ ]:
#grouped_df = (
    df
    .groupby(["List Year", "Town", "Sale Price Group"], observed=True)
    .agg(
        Avg_Sales_Ratio=("Sales Ratio", "mean"),
        Count_Sales=("Sales Ratio", "count"),
        Total_Sales_Amount=("Sale Amount", "sum"),
        Avg_Sale_Amount=("Sale Amount", "mean")
    )
    .reset_index()
)

# Optional: sort results
grouped_df = grouped_df.sort_values(
    by=["Sale Price Group", "Avg_Sales_Ratio"],
    ascending=[True, False]
)

#grouped_df



In [ ]:
pd.set_option('display.float_format', '{:,.0f}'.format)


In [ ]:
# 1️⃣ Make sure 'Sale Price Group' is categorical
if not pd.api.types.is_categorical_dtype(df["Sale Price Group"]):
    df["Sale Price Group"] = df["Sale Price Group"].astype("category")

# 2️⃣ Add missing categories only if needed
new_cats = ["<300k", "300k-800k", "800k+"]
df["Sale Price Group"] = df["Sale Price Group"].cat.add_categories(
    [c for c in new_cats if c not in df["Sale Price Group"].cat.categories]
)

# 3️⃣ Groupby for stats
base_stats = (
    df.groupby(["List Year", "Town"], observed=True)
      .agg(
          Avg_Sale_Amount=("Sale Amount", "mean"),
          Count_Sales=("Sale Price Group", "count"),
          Avg_Sales_Ratio=("Sales Ratio", "mean"),
          Total_Sales=("Sale Amount", "sum")
      )
      .reset_index()
)

# 4️⃣ Count per Sale Price Group
count_per_group = (
    df.groupby(["List Year", "Town", "Sale Price Group"], observed=True)
      .size()
      .reset_index(name="Count_Per_Group")
)

# 5️⃣ Merge together
result = base_stats.merge(count_per_group, on=["List Year", "Town"], how="left")



In [ ]:
result

In [ ]:
result.shape

In [ ]:
result.to_csv("re_town_levels.csv")

In [2]:
import os
print(os.getcwd())

C:\Users\mrsmo\Real Estate Sales Project
